#데이터 수집
프렌즈 대본을 워드 파일로 구한 수 텍스트 파일로 변환하여 사용하였다. 
https://pfe2000.tistory.com/266



# 데이터 전처리

colab으로 작성한 코드입니다

In [1]:
!git clone https://github.com/TMTsProject/django-chatbot.git

Cloning into 'django-chatbot'...
remote: Enumerating objects: 609, done.
remote: Counting objects: 100% (609/609), done.
remote: Compressing objects: 100% (397/397), done.
remote: Total 609 (delta 280), reused 495 (delta 191), pack-reused 0
Receiving objects: 100% (609/609), 30.38 MiB | 30.50 MiB/s, done.
Resolving deltas: 100% (280/280), done.


In [4]:
%cd django-chatbot/

/content/django-chatbot


In [5]:
# 라이브러리 import
import pandas as pd
import re
import os

In [ ]:
# path는 프렌즈 대본을 가리키도록 환경에 맞게 설정
PATH = 'Data/friends_raw_data/'

# 프렌즈 대본 폴더의 파일 이름을 모두 읽어옴
file_list = os.listdir(PATH)

# 파일 이름 정렬
file_list = sorted(file_list)       # 시즌1, 시즌10, 시즌2, 시즌3, ... 순서로 정렬
file_list.append(file_list.pop(1))  # 시즌 10.txt를 맨 뒤로 보냄

# 전처리한 데이터를 저장할 데이터프레임 생성
# 장면 번호 : 캐릭터 : 대사 형식
data = pd.DataFrame(columns=['Scene', 'Character', 'Line'], dtype=str)
data['Scene'] = data['Scene'].astype(int)

scene_num = 0

for f_name in file_list:
    with open(PATH+f_name, 'r') as f:
        lines = f.readlines()
        for line in lines:
            # Rachel : Hi! 처럼 대사구분이 ':'로 되어있으므로 이를 기준으로 캐릭터명과 대사를 구분함
            if ':' in line:
                script = line.strip().split(':')
                # :가 여러번 나오면 뒤는 대사에 포함되게 함.
                if (script[0].lower()[:6] == '[scene') or (script[0].lower()[:6] == '(scene'):
                    scene_num += 1
                data = data.append({'Scene': scene_num, 'Character' : script[0].lower(), 'Line':(' ').join(script[1:])},ignore_index=True)

# 대사가 아닌 부분, 즉 지문이나 설명, 나레이션은 삭제함
idxList = []
for idx in range(len(data['Character'])):
    if 'by' in data['Character'][idx] :
        idxList.append(idx)
    elif 'note' in data['Character'][idx] :
        idxList.append(idx)
    elif 'scene' in data['Character'][idx] :
        idxList.append(idx)
    elif 'with' in data['Character'][idx] :
        idxList.append(idx)
    elif data['Character'][idx][0] == '{':
        idxList.append(idx)
    elif data['Character'][idx][0] == '[':
        idxList.append(idx)
    elif data['Character'][idx][0] == '(':
        idxList.append(idx)
    elif data['Character'][idx] == 'cut to' :
        idxList.append(idx)
    elif data['Character'][idx] == 'narrator' :
        idxList.append(idx)

data.drop(idxList , inplace=True)
data = data.reset_index(drop=True)

# 캐릭터명 통일
# 캐릭터명에 오타가 있는 경우, 대문자/소문자가 다른 경우, 이름을 줄인 경우 그리고 이름에 괄호가 있는 경우를 없애고 모두 하나로 통일시킴
# Ex. Rach -> rachel, RACHEL -> rachel 등
for i in range(len(data['Character'])):
    if 'and' in data['Character'][i]:
        continue
    if "ross " in data['Character'][i]:
        data['Character'][i] = "ross" 
    elif "rachel " in data['Character'][i]:
        data['Character'][i] = "rachel"
    elif "chandler " in data['Character'][i]:
        data['Character'][i] = "chandler"
    elif "monica " in data['Character'][i]:
        data['Character'][i] = "monica" 
    elif "phoebe " in data['Character'][i]:
        data['Character'][i] = "phoebe" 
    elif "joey " in data['Character'][i]:
        data['Character'][i] = "joey"
    elif "joey's voice" in data['Character'][i]:
        data['Character'][i] = "joey"
    elif "amy " in data['Character'][i]:
        data['Character'][i] = "amy"

for idx in range(len(data['Line'])):
    data['Character'][idx] = re.sub('\(.*\)','', data['Character'][idx])
    data['Character'][idx] = re.sub('\[.*\]','', data['Character'][idx])
    data['Character'][idx] = re.sub('  ', ' ', data['Character'][idx])
    data['Character'][idx] = data['Character'][idx].strip()
    data['Line'][idx] = re.sub('\(.*\)','', data['Line'][idx])
    data['Line'][idx] = re.sub('\[.*\]','', data['Line'][idx])
    data['Line'][idx] = re.sub('  ', ' ', data['Line'][idx])
    data['Line'][idx] = data['Line'][idx].strip()

data['Character'][data['Character']== "rachel/ross"] = "rachel"
data['Character'][data['Character']== "rach"] = "rachel"
data['Character'][data['Character']== "rahcel"] = "rachel"
data['Character'][data['Character']== "racel"] = "rachel"	
data['Character'][data['Character']== "chan"] = "chandler"
data['Character'][data['Character']== "mnca"] = "monica"
data['Character'][data['Character']== "phoe"] = "phoebe"

indexNames = data[ data['Line'] == '' ].index
data.drop(indexNames , inplace=True)
data = data.reset_index(drop=True)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:65: SettingWithCopyWarning: 
A value is trying to b

In [ ]:
data.to_csv(PATH+'Friends_new.csv',index=False)

In [ ]:
data

,Scene,Character,Line
0,1,monica,There's nothing to tell! He's just some guy I ...
1,1,joey,"C'mon, you're going out with the guy! There's ..."
2,1,chandler,"All right Joey, be nice. So does he have a hum..."
3,1,phoebe,"Wait, does he eat chalk?"
4,1,phoebe,"Just, 'cause, I don't want her to go through w..."
...,...,...,...
61589,3097,chandler,"Oh, it's gonna be okay."
61590,3097,rachel,Do you guys have to go to the new house right ...
61591,3097,monica,We got some time.
61592,3097,rachel,"Okay, should we get some coffee?"


In [ ]:
data['Character'].value_counts()

rachel                                                               9324
ross                                                                 9107
chandler                                                             8480
monica                                                               8423
joey                                                                 8176
phoebe                                                               7579
mike                                                                 357 
all                                                                  333 
richard                                                              256 
mr. geller                                                           217 
janice                                                               214 
emily                                                                202 
carol                                                                192 
charlie                               